## Multi-Agenten-System mit LangGraph (Whisper, NeMo, Matching & Themenextraktion)

In diesem Workshop bauen wir ein **Multi-Agenten-System** mithilfe von **LangGraph**. Folgende Agenten (Nodes) kommen zum Einsatz:

1. **Whisper-Agent**: Führt Speech-to-Text mit Whisper durch.
2. **Pyannote-Agent**: Führt die Sprechertrennung durch.
3. **Matching-Agent**: Verknüpft die Ergebnisse von Whisper und Pyannote.
4. **Summarization-Agent**: Fasst den gefundenen Dialog/Gesprächsinhalte zusammen.
5. **Topic-Agent**: Extrahiert ein **Thema** oder Kern-Aussage aus dem Gespräch.
6. **Search-Agent**: Frag das Internet nach diesem Thema (z. B. via Tavily) und holt relevante Informationen.

**Ziel**: Am Ende haben wir einen Workflow, bei dem eine Audiodatei in ein Protokoll gewandelt wird, wir eine **Zusammenfassung** erhalten, ein **Thema** extrahieren und dazu eine **Internetrecherche** durchführen.


### 1. Imports

In [1]:
import os
from dotenv import load_dotenv
from langgraph.graph import START, StateGraph, END
from typing_extensions import TypedDict
from IPython.display import Image, display
from tavily import TavilyClient
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from foundation.helper import run_whisper, run_pyannote, run_matching

### 2. Laden der Umgebungsvariablen und erstellen der Chat Modelle

In [2]:
load_dotenv()

PYANNOTE_HF_TOKEN = os.getenv("PYANNOTE_HF_TOKEN", "")
PYANNOTE_MODEL = os.getenv("PYANNOTE_MODEL", "PYANNOTE_MODEL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://host.docker.internal:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2:1b")
STACKIT_VLLM_API_KEY = os.getenv("STACKIT_VLLM_API_KEY", "")
STACKIT_MODEL = os.getenv("STACKIT_MODEL", "neuralmagic/Meta-Llama-3.1-70B-Instruct-FP8")
STACKIT_BASE_URL = os.getenv("STACKIT_BASE_URL", "https://api.openai-compat.model-serving.eu01.onstackit.cloud/v1")


stackit_llm = ChatOpenAI(
    model=STACKIT_MODEL,
    base_url=STACKIT_BASE_URL,
    api_key=STACKIT_VLLM_API_KEY,
)

ollama_llm = ChatOllama(
    model=OLLAMA_MODEL,
    base_url=OLLAMA_BASE_URL,
    temperature=0.8
)


### 3. State definieren


In [3]:
class MyState(TypedDict):
    audio_path: str
    # ...
    internet_search_result: str

### 4. Node definieren

In [4]:
# Node
def whisper(state: MyState):
    result = run_whisper(audio_path=state["audio_path"])
    return {"whisper_result": result}

# Node
def pyannote(state: MyState):
    pass

# Node
def matcher(state: MyState):
   pass

# Node
def summarize(state: MyState):
    messages_for_llm = [
        ]
    pass

# Node
def extract_topic(state: MyState):
    messages_for_llm = [
        AIMessage(content="You are a helpful assistant and an expert in extracting the main topic of the conversation out of a summary."),
        HumanMessage(content=f"Please extract one single topic out of this summary: {state["summary"]}")
    ]

    topic = stackit_llm.invoke(messages_for_llm)
    return {"topic": topic.content}

# Node
def internet_search(state: MyState):
    pass



### 5. Graph definieren

In [5]:
# StateGraph initialisieren
graph = StateGraph(MyState)

# Agent (Node) hinzufügen
graph.add_node("Whisper", whisper)
# ...
graph.add_node("Internet Researcher", internet_search)

# Ablauf im Graph definieren durch Edges
graph.add_edge(START, "Whisper")
# ...
graph.add_edge("Internet Researcher", END)

# Graph erstellen
graph = graph.compile()


### 6. Graph visualisieren

In [ ]:
# Graph anzeigen lassen
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

### 7. Graph aufrufen

In [ ]:
# Graph aufrufen
audio_path = "./foundation/audios/Sonic-Manus-Pitch.mp3"

result_state = graph.invoke({"audio_path": audio_path}, debug=True)

### 8. Ergebnisse ausgeben

In [ ]:
# ...